In [ ]:
!pip install duckduckgo_search

In [ ]:
from duckduckgo_search import DDGS

def search_images(keyword, max_results=10):
    with DDGS() as ddgs:
        images = ddgs.images(
            keyword,
            max_results=max_results
        )
        return [img['image'] for img in images]

In [ ]:
keyword_1 = "shark"
image_urls_1 = search_images(keyword_1, 250)
print(len(image_urls_1))

keyword_2 = "deer"
image_urls_2 = search_images(keyword_2, 250)
print(len(image_urls_2))

keyword_3 = "rabbit"
image_urls_3 = search_images(keyword_3, 250)
print(len(image_urls_3))

keyword_4 = "dog"
image_urls_4 = search_images(keyword_4, 250)
print(len(image_urls_4))

250
250
250
250


In [ ]:
import os
import requests
from urllib.parse import urlparse
import warnings

def download_image(url, folder, custom_name=None, verbose=True):
    # Create the folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)

    # Get the filename from the URL or use the custom name
    if custom_name:
        filename = custom_name
    else:
        filename = os.path.basename(urlparse(url).path)
        if not filename:
            filename = 'image.jpg'  # Default filename if none is found in the URL

    # Ensure the filename has an extension
    if not os.path.splitext(filename)[1]:
        filename += '.jpg'

    filepath = os.path.join(folder, filename)

    # If the file already exists, append a number to make it unique
    base, extension = os.path.splitext(filepath)
    counter = 1
    while os.path.exists(filepath):
        filepath = f"{base}_{counter}{extension}"
        counter += 1

    try:
        # Send a GET request to the URL with a timeout of 10 seconds
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Check if the content type is an image
        content_type = response.headers.get('content-type', '')
        if not content_type.startswith('image'):
            if verbose:
                warnings.warn(f"The URL does not point to an image. Content-Type: {content_type}")
            return False

        # Write the image content to the file
        with open(filepath, 'wb') as f:
            f.write(response.content)

        if verbose:
            print(f"Image successfully downloaded: {filepath}")
        return True

    except requests.exceptions.Timeout:
        if verbose:
            warnings.warn(f"Download timed out for URL: {url}")
    except requests.exceptions.HTTPError as e:
        if verbose:
            warnings.warn(f"HTTP error occurred: {e}")
    except requests.exceptions.RequestException as e:
        if verbose:
            warnings.warn(f"An error occurred while downloading the image: {e}")
    except IOError as e:
        if verbose:
            warnings.warn(f"An error occurred while writing the file: {e}")

    return False

In [ ]:
from tqdm.notebook import tqdm

for i, url in enumerate(tqdm(image_urls_1)):
    download_image(url, "./dataset/shark/", f'image{i}.jpg', verbose=False)

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
from tqdm.notebook import tqdm

for i, url in enumerate(tqdm(image_urls_2)):
    download_image(url, "./dataset/deer/", f'image{i+250}.jpg', verbose=False)

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
from tqdm.notebook import tqdm

for i, url in enumerate(tqdm(image_urls_3)):
    download_image(url, "./dataset/rabbit/", f'image{i+500}.jpg', verbose=False)

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
from tqdm.notebook import tqdm

for i, url in enumerate(tqdm(image_urls_4)):
    download_image(url, "./dataset/dog/", f'image{i+750}.jpg', verbose=False)

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
# ...
from torchvision import datasets, transforms
# ...

# Data transformations
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

# Load the dataset
dataset = datasets.ImageFolder(root='./dataset', transform=transform)

In [ ]:
import torch

train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
%%capture
!pip install lightning

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import lightning as L

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# if using windows use next code instead
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: mps


In [ ]:
# load the training data
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

# load the test data
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
class NN(L.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate

    def _common_eval(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._common_eval(batch, batch_idx)
        self.log('train_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self._common_eval(batch, batch_idx)
        self.log('test_loss', loss)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
def model1(n_features, in_channels, dropout, kernel_size, stride, padding, **kwargs):
    return [
        nn.Conv2d(
            in_channels=in_channels,
            out_channels=16,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=16,
            out_channels=64,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        ),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(in_features=64*28*28, out_features=128),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(in_features=128, out_features=n_features)
    ]

model2 = lambda n_features, in_channels, dropout, kernel_size, stride, padding: [
    nn.Conv2d(
        in_channels=in_channels,
        out_channels=64,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
    ),
    nn.ReLU(),
    nn.Conv2d(
        in_channels=64,
        out_channels=32,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
    ),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(in_features=28*28*32, out_features=128),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(in_features=128, out_features=n_features)
]

model3 = lambda n_features, in_channels, dropout, kernel_size, stride, padding: [
    nn.Conv2d(
        in_channels=in_channels,
        out_channels=32,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
    ),
    nn.ReLU(),
    nn.Conv2d(
        in_channels=32,
        out_channels=64,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
    ),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(in_features=28*28*64, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=64),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(in_features=64, out_features=n_features)
]

def get_model(model, epochs=None, learning_rate=None, **kwargs):
    return nn.Sequential(*model(**kwargs))

In [ ]:
def evaluate(model, test_loader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in test_loader:
            # Move images and labels to the device
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'learning_rate': [1e-4, 5e-4, 1e-3, 5e-3],
    'model': [
        # model1,
        model2,
        model3,
    ],
    'epochs': [5, 10],
    'n_features': [10],
    'in_channels': [3],
    'dropout': [0.1, 0.25, 0.5],
    'kernel_size': [(3, 3)],
    'stride': [1],
    'padding': [1],
}

grid = ParameterGrid(param_grid)

In [ ]:
import logging
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)


In [ ]:
results = []

In [ ]:
import time

for params in grid:
    start = time.time()
    model = NN(
        model=get_model(**params),
        learning_rate=params['learning_rate']
    ).to(device)
    print(id(model))
    trainer = L.Trainer(max_epochs=params['epochs'], enable_model_summary=False)
    trainer.fit(model=model, train_dataloaders=train_loader)
    acc = evaluate(model.model.to(device), test_loader)
    end = time.time()
    print(f'Accuracy: {acc} in {end - start} s with params: {params}', flush=True)
    results.append((acc, params))


11137913952


/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(l

Training: |                                               | 0/? [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/PIL/Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Accuracy: 0.5145631067961165 in 77.4625768661499 s with params: {'dropout': 0.1, 'epochs': 5, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x297aa5da0>, 'n_features': 10, 'padding': 1, 'stride': 1}
13525812576


...

Accuracy: 0.4854368932038835 in 158.16828203201294 s with params: {'dropout': 0.5, 'epochs': 10, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x297aa4540>, 'n_features': 10, 'padding': 1, 'stride': 1}
11686969856


Training: |                                               | 0/? [00:00<?, ?it/s]

Accuracy: 0.3883495145631068 in 157.22654700279236 s with params: {'dropout': 0.5, 'epochs': 10, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x297aa5da0>, 'n_features': 10, 'padding': 1, 'stride': 1}
13525862784


Training: |                                               | 0/? [00:00<?, ?it/s]

Accuracy: 0.4854368932038835 in 157.64935398101807 s with params: {'dropout': 0.5, 'epochs': 10, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x297aa4540>, 'n_features': 10, 'padding': 1, 'stride': 1}


In [ ]:
print('\n'.join(map(str, sorted(results, reverse=True, key=lambda x: x[0]))))

(0.5728155339805825, {'dropout': 0.25, 'epochs': 10, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x297aa4540>, 'n_features': 10, 'padding': 1, 'stride': 1})
(0.5728155339805825, {'dropout': 0.5, 'epochs': 10, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x297aa5da0>, 'n_features': 10, 'padding': 1, 'stride': 1})
(0.5631067961165048, {'dropout': 0.25, 'epochs': 10, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x297aa5da0>, 'n_features': 10, 'padding': 1, 'stride': 1})
(0.5631067961165048, {'dropout': 0.5, 'epochs': 10, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x297aa5da0>, 'n_features': 10, 'padding': 1, 'stride': 1})
(0.5485436893203883, {'dropout': 0.1, 'epochs': 10, 'in_channels': 3, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x297aa4540>, '